In [ ]:
import atexit
from datetime import datetime, timedelta
from kafka import KafkaProducer
from time import sleep
import requests

citeUrls = ('http://api.openweathermap.org/data/2.5/weather?q=Warszawa&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Kraków&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?id=3093133&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Wrocław&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Poznań&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Gdańsk&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Szczecin&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Bydgoszcz&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?id=765876&appid={your_api_key}', \
            'http://api.openweathermap.org/data/2.5/weather?q=Białystok&appid={your_api_key}',)

counter = 0

producer = KafkaProducer(bootstrap_servers=('master','slave01'))

while True:
    index = counter % 10
    url = citeUrls[index]
    try:
        data = requests.get(url).json()
        city_name = data['name']
        timestamp = data['dt']
        ms_timestamp = int(round(timestamp * 1000))
        main_obj = tuple(data['main'].values())
        temp = main_obj[0]
        feels_like = main_obj[1]

        #Połączenie temperatury aktualnej z temperaturą odcz. 
        value = str(temp) + '|' + str(feels_like)

        producer.send('final_open_weather_mw_v2', value.encode(), city_name.encode(), timestamp_ms=ms_timestamp)

        print(f'Row: Value = {value} ||| kay = {city_name} ||| timestamp = {ms_timestamp}')
    except Exception as e:
        print(f'{e}, {data}')
    
    counter += 1
    if 9 == index:
        producer.flush()
    
    sleep(360)
    

#Debugowanie wystarczy umiescic pod koniec bloku TRY:     
    #print(f'{ms_timestamp}')
    #print(f'{type(ms_timestamp)}')
    #print(f'{city_name}')
    #print(f'{type(city_name)}')
    #print(f'{temp}')
    #print(f'{type(temp)}')
    #print(f'{feels_like}')
    #print(f'{type(feels_like)}')
    #print(f'{value}')
    #print(f'{type(value)}')



Row: Value = 287.07|283.38 ||| kay = Warsaw ||| timestamp = 1590344108000
Row: Value = 283.35|279.59 ||| kay = Krakow ||| timestamp = 1590344378000
Row: Value = 283.74|280.53 ||| kay = Łódź ||| timestamp = 1590344347000
Row: Value = 285.25|282.73 ||| kay = Wrocław ||| timestamp = 1590344404000
Row: Value = 286.52|283.69 ||| kay = Poznań ||| timestamp = 1590344459000
Row: Value = 285.56|282.76 ||| kay = Gdańsk ||| timestamp = 1590344420000
